In [6]:
import wrds
import pandas as pd
import pytz
import datetime as dt
import pandas_market_calendars as mcal

In [7]:
db = wrds.Connection(wrds_username = "connorwz")

Loading library list...
Done


In [19]:
def SP500_data(year_range):
    first_year, last_year = str(year_range[0]),str(year_range[1])
    sp_500_query = f"""SELECT a.*, b.date, b.ret, b.prc, b.openprc
                        FROM crsp.dsp500list as a,
                        crsp.dsf as b
                        WHERE a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/{first_year}' and b.date<='12/31/{last_year}'
                        order by date;"""
    sp_500 = db.raw_sql(sp_500_query,date_cols=['start', 'ending', 'date'])
    dse = db.raw_sql("""
                        select comnam,ncusip, namedt, nameendt,permno
                        from crsp.dsenames
                        """, date_cols=['namedt', 'nameendt'])
    dse['nameendt']=dse['nameendt'].fillna(pd.to_datetime('today'))
    sp500_full = pd.merge(sp_500, dse, how = 'left', on = 'permno')

    sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                                & (sp500_full.date<=sp500_full.nameendt)]
    sp500_full.reset_index(inplace = True,drop = True)
    sp500_full = sp500_full[["permno","date","comnam","ret","openprc","prc"]]
    sp500_full["prc"] = sp500_full["prc"].apply(abs)
    sp500_full["CO_ret"] = (sp500_full['prc'] - sp500_full['openprc'])/sp500_full['openprc']
    return sp500_full
